# Загрузка данных, импорт библиотек и настройка окружения

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
categories_path = '/content/drive/MyDrive/maxidom_contest/data/categories.csv'
offers_expanded_path = '/content/drive/MyDrive/maxidom_contest/data/offers_expanded.csv'
promos_path = '/content/drive/MyDrive/maxidom_contest/data/promos.csv'
offers_path = '/content/drive/MyDrive/maxidom_contest/data/offers.csv'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
offers_expanded = pd.read_csv(offers_path)
categories = pd.read_csv(categories_path)
promos = pd.read_csv(promos_path)
offers = pd.read_csv(offers_path)


# Data Quality

### Структура данных - offers

In [ ]:
print("\nТипы данных:")
print(offers.dtypes)
print("\nИнформация:")
offers.info()


Типы данных:
offer_id                 int64
available                 bool
url                     object
price                  float64
currency                object
category_id              int64
picture                 object
sales_notes             object
delivery                  bool
local_delivery_cost    float64
name                    object
vendor                  object
country_of_origin       object
description             object
market_description      object
weight                 float64
params                  object
dtype: object

Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64490 entries, 0 to 64489
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   offer_id             64490 non-null  int64  
 1   available            64490 non-null  bool   
 2   url                  64490 non-null  object 
 3   price                64490 non-null  float64
 4   currency          

### Структура данных - categories

In [ ]:
print("\nТипы данных:")
print(categories.dtypes)
print("\nИнформация:")
categories.info()


Типы данных:
category_id      int64
parent_id      float64
name            object
dtype: object

Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3231 entries, 0 to 3230
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   category_id  3231 non-null   int64  
 1   parent_id    3204 non-null   float64
 2   name         3231 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 75.9+ KB


### Структура данных - promos

In [ ]:
print("\nТипы данных:")
print(promos.dtypes)
print("\nИнформация:")
promos.info()


Типы данных:
promo_id            int64
promo_type         object
start_date         object
end_date           object
description        object
url                object
offer_id            int64
discount_price    float64
currency           object
dtype: object

Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64437 entries, 0 to 64436
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   promo_id        64437 non-null  int64  
 1   promo_type      64437 non-null  object 
 2   start_date      64437 non-null  object 
 3   end_date        64437 non-null  object 
 4   description     64437 non-null  object 
 5   url             64437 non-null  object 
 6   offer_id        64437 non-null  int64  
 7   discount_price  64437 non-null  float64
 8   currency        64437 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 4.4+ MB


### DQ анализ

In [ ]:
def data_quality_report(df, df_name):

    print(f"Размер датасета: {df.shape[0]:,} строк, {df.shape[1]} колонок")
    print(f"Использование памяти: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # ПРОПУЩЕННЫЕ
    print("\n" + "─" * 80)
    print("  ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ")
    print("─" * 80)

    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100

    missing_df = (
        pd.DataFrame({
            "Колонка": df.columns,
            "Пропуски": missing.values,
            "Процент": missing_pct.values
        })
        .sort_values("Пропуски", ascending=False)
    )

    if missing_df["Пропуски"].sum() > 0:
        print(missing_df[missing_df["Пропуски"] > 0].to_string(index=False))
    else:
        print("Пропущенных значений не обнаружено")

    # ДУБЛИКАТЫ
    print("\n" + "─" * 80)
    print("  ДУБЛИКАТЫ")
    print("─" * 80)

    duplicates = df.duplicated().sum()
    duplicates_pct = (duplicates / len(df)) * 100

    print(f"Всего дубликатов: {duplicates:,} ({duplicates_pct:.2f}%)")

    if duplicates > 0:
        print("\nПримеры дубликатов:")
        print(df[df.duplicated(keep=False)].head(10))

    # УНИКАЛЬНЫЕ
    print("\n" + "─" * 80)
    print("  УНИКАЛЬНЫЕ ЗНАЧЕНИЯ")
    print("─" * 80)

    uniqueness = []
    for col in df.columns:
        unique_count = df[col].nunique()
        unique_pct = (unique_count / len(df)) * 100

        uniqueness.append({
            "Колонка": col,
            "Уникальных": unique_count,
            "Процент": f"{unique_pct:.2f}%",
            "Тип": str(df[col].dtype)
        })

    uniqueness_df = pd.DataFrame(uniqueness)
    print(uniqueness_df.to_string(index=False))

    # ЧИСЛОВЫЕ КОЛОНКИ
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    if len(numeric_cols) > 0:
        print("\n" + "─" * 80)
        print("  СТАТИСТИКА ЧИСЛОВЫХ КОЛОНОК")
        print("─" * 80)

        print(df[numeric_cols].describe().T)

        print("\nПроверка на выбросы (IQR):")

        for col in numeric_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1

            outliers = df[
                (df[col] < Q1 - 1.5 * IQR) |
                (df[col] > Q3 + 1.5 * IQR)
            ][col]

            if len(outliers) > 0:
                print(
                    f"  {col}: {len(outliers)} выбросов "
                    f"({len(outliers) / len(df) * 100:.2f}%)"
                )

     # ТЕКСТОВЫЕ КОЛОНКИ

    text_cols = df.select_dtypes(include=["object"]).columns
    skip_cols = ["url", "picture", "params"]
    meaningful_text_cols = [col for col in text_cols if col not in skip_cols]

    if len(meaningful_text_cols) > 0:

        print("\n" + "─" * 80)
        print("  СТАТИСТИКА ТЕКСТОВЫХ КОЛОНОК")
        print("─" * 80)

        for col in meaningful_text_cols:
            print(f"\n{col}:")
            print(f"  Уникальных значений: {df[col].nunique()}")

            unique_count = df[col].nunique()

            if unique_count <= 5:
                print("  Распределение значений:")
                print(df[col].value_counts())

            elif unique_count <= 20:
                print("  Топ-10 значений:")
                print(df[col].value_counts().head(10))

    return missing_df


In [ ]:
quality_offers = data_quality_report(offers, "OFFERS")

Размер датасета: 64,490 строк, 17 колонок
Использование памяти: 192.43 MB

────────────────────────────────────────────────────────────────────────────────
  ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
           Колонка  Пропуски  Процент
market_description       827 1.282369

────────────────────────────────────────────────────────────────────────────────
  ДУБЛИКАТЫ
────────────────────────────────────────────────────────────────────────────────
Всего дубликатов: 0 (0.00%)

────────────────────────────────────────────────────────────────────────────────
  УНИКАЛЬНЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
            Колонка  Уникальных Процент     Тип
           offer_id       64490 100.00%   int64
          available           1   0.00%    bool
                url       64490 100.00%  object
              price        6520  10.11% float64
           currency           1   0.

In [ ]:
quality_categories = data_quality_report(categories, "CATEGORIES")

Размер датасета: 3,231 строк, 3 колонок
Использование памяти: 0.52 MB

────────────────────────────────────────────────────────────────────────────────
  ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
  Колонка  Пропуски  Процент
parent_id        27 0.835655

────────────────────────────────────────────────────────────────────────────────
  ДУБЛИКАТЫ
────────────────────────────────────────────────────────────────────────────────
Всего дубликатов: 0 (0.00%)

────────────────────────────────────────────────────────────────────────────────
  УНИКАЛЬНЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
    Колонка  Уникальных Процент     Тип
category_id        3231 100.00%   int64
  parent_id         438  13.56% float64
       name        3209  99.32%  object

────────────────────────────────────────────────────────────────────────────────
  СТАТИСТИКА ЧИСЛОВЫХ КОЛОНОК
──────────────────────────

In [ ]:
quality_promos = data_quality_report(promos, "PROMOS")

Размер датасета: 64,437 строк, 9 колонок
Использование памяти: 28.02 MB

────────────────────────────────────────────────────────────────────────────────
  ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
Пропущенных значений не обнаружено

────────────────────────────────────────────────────────────────────────────────
  ДУБЛИКАТЫ
────────────────────────────────────────────────────────────────────────────────
Всего дубликатов: 0 (0.00%)

────────────────────────────────────────────────────────────────────────────────
  УНИКАЛЬНЫЕ ЗНАЧЕНИЯ
────────────────────────────────────────────────────────────────────────────────
       Колонка  Уникальных Процент     Тип
      promo_id           1   0.00%   int64
    promo_type           1   0.00%  object
    start_date           1   0.00%  object
      end_date           1   0.00%  object
   description           1   0.00%  object
           url           1   0.00%  object
      offer_id    

### Анализ связей между таблицами

In [ ]:
if 'category_id' in offers.columns and 'category_id' in categories.columns:
    print("\nOFFERS и CATEGORIES")
    print("─" * 80)

    offers_with_cat = offers.merge(categories, on='category_id', how='left', indicator=True)

    print(f"Всего товаров в offers: {len(offers):,}")
    print(f"Товаров с найденными категориями: {(offers_with_cat['_merge'] == 'both').sum():,}")
    print(f"Товаров без категорий: {(offers_with_cat['_merge'] == 'left_only').sum():,}")

    orphan_categories = set(offers['category_id'].dropna()) - set(categories['category_id'].dropna())
    print(f"Несуществующих category_id в offers: {len(orphan_categories)}")
    if len(orphan_categories) > 0 and len(orphan_categories) < 20:
        print(f"Примеры: {list(orphan_categories)[:10]}")

if 'offer_id' in offers.columns and 'offer_id' in promos.columns:
    print("\nOFFERS и PROMOS")
    print("─" * 80)

    offers_in_promos = set(promos['offer_id'].dropna())
    offers_in_offers = set(offers['offer_id'].dropna())

    print(f"Всего товаров в offers: {len(offers):,}")
    print(f"Товаров с промо: {len(offers_in_promos):,}")
    print(f"Процент товаров с промо: {len(offers_in_promos) / len(offers) * 100:.2f}%")

    missing_offers = offers_in_promos - offers_in_offers
    print(f"Товаров из promos, которых нет в offers: {len(missing_offers)}")
    if len(missing_offers) > 0 and len(missing_offers) < 20:
        print(f"Примеры: {list(missing_offers)[:10]}")


OFFERS и CATEGORIES
────────────────────────────────────────────────────────────────────────────────
Всего товаров в offers: 64,490
Товаров с найденными категориями: 64,490
Товаров без категорий: 0
Несуществующих category_id в offers: 0

OFFERS и PROMOS
────────────────────────────────────────────────────────────────────────────────
Всего товаров в offers: 64,490
Товаров с промо: 64,437
Процент товаров с промо: 99.92%
Товаров из promos, которых нет в offers: 0


### Бизнес-ориентированные проверки данных

In [ ]:
if 'available' in offers.columns:
    print("\nДОСТУПНОСТЬ ТОВАРОВ")
    print("─" * 80)
    availability = offers['available'].value_counts()
    print(availability)
    total = len(offers)
    for val, count in availability.items():
        print(f"{val}: {count:,} ({count/total*100:.2f}%)")

if 'price' in offers.columns:
    print("\nАНАЛИЗ ЦЕН")
    print("─" * 80)
    prices = offers['price'].dropna()
    print(f"Товаров с ценой: {len(prices):,}")
    print(f"Товаров без цены: {offers['price'].isna().sum():,}")
    print(f"Минимальная цена: {prices.min():.2f}")
    print(f"Максимальная цена: {prices.max():.2f}")
    print(f"Средняя цена: {prices.mean():.2f}")
    print(f"Медианная цена: {prices.median():.2f}")

    zero_prices = (offers['price'] == 0).sum()
    if zero_prices > 0:
        print(f"\nТоваров с нулевой ценой: {zero_prices:,}")

if 'currency' in offers.columns:
    print("\nВАЛЮТЫ")
    print("─" * 80)
    print(offers['currency'].value_counts())

if 'vendor' in offers.columns:
    print("\nПРОИЗВОДИТЕЛИ (Топ-20)")
    print("─" * 80)
    print(offers['vendor'].value_counts().head(20))

if 'parent_id' in categories.columns:
    print("\nИЕРАРХИЯ КАТЕГОРИЙ")
    print("─" * 80)
    root_categories = categories[categories['parent_id'].isna()]
    print(f"Корневых категорий: {len(root_categories)}")
    print(f"Категорий с родителем: {len(categories) - len(root_categories)}")

    max_depth = 0
    for cat_id in root_categories['category_id'].values:
        depth = 0
        current_level = [cat_id]
        while current_level:
            depth += 1
            next_level = []
            for c_id in current_level:
                children = categories[categories['parent_id'] == c_id]['category_id'].values
                next_level.extend(children)
            current_level = next_level
        max_depth = max(max_depth, depth)

    print(f"Максимальная глубина дерева категорий: {max_depth}")

if 'promo_type' in promos.columns:
    print("\nТИПЫ ПРОМО-АКЦИЙ")
    print("─" * 80)
    print(promos['promo_type'].value_counts())

if 'discount_price' in promos.columns and 'offer_id' in promos.columns:
    print("\nСТАТИСТИКА СКИДОК")
    print("─" * 80)

    promos_with_offers = promos.merge(
        offers[['offer_id', 'price']],
        on='offer_id',
        how='left'
    )

    promos_with_offers['discount_amount'] = promos_with_offers['price'] - promos_with_offers['discount_price']
    promos_with_offers['discount_percent'] = (promos_with_offers['discount_amount'] / promos_with_offers['price']) * 100

    valid_discounts = promos_with_offers['discount_percent'].dropna()
    if len(valid_discounts) > 0:
        print(f"Средняя скидка: {valid_discounts.mean():.2f}%")
        print(f"Медианная скидка: {valid_discounts.median():.2f}%")
        print(f"Минимальная скидка: {valid_discounts.min():.2f}%")
        print(f"Максимальная скидка: {valid_discounts.max():.2f}%")



ДОСТУПНОСТЬ ТОВАРОВ
────────────────────────────────────────────────────────────────────────────────
available
True    64490
Name: count, dtype: int64
True: 64,490 (100.00%)

АНАЛИЗ ЦЕН
────────────────────────────────────────────────────────────────────────────────
Товаров с ценой: 64,490
Товаров без цены: 0
Минимальная цена: 4.00
Максимальная цена: 209990.00
Средняя цена: 2617.92
Медианная цена: 599.00

ВАЛЮТЫ
────────────────────────────────────────────────────────────────────────────────
currency
RUB    64490
Name: count, dtype: int64

ПРОИЗВОДИТЕЛИ (Топ-20)
────────────────────────────────────────────────────────────────────────────────
vendor
Нет марки      11385
MAXIJOY         1869
ЕВРОПАРТНЕР     1268
ЭРА              927
TODELIA          820
UGO LOKS         681
УЮТ              670
NAVIGATOR        666
AMBRELLA         609
KOOPMAN          552
РЫЖИЙ КОТ        456
VITARTA          427
SALAG            408
ПОИСК            393
MALLONY          390
СТРОЙБАТ         366
VENMEB

# Новый раздел